In [1]:
#!pip install torch==1.13.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers==4.30.0
!pip install datasets==2.13.2
!pip install accelerate -U
!pip install evaluate
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
# system packages
from pathlib import Path
import shutil
import urllib
import tarfile
import sys
import os
# data and numerical management packages
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
# useful during debugging (progress bars)
from tqdm import tqdm
from transformers import set_seed

seed = 852

random.seed(seed)
np.random.seed(seed)
set_seed(seed)

In [3]:
pwd

'/content'

In [5]:
dataset_folder = Path.cwd().joinpath("MELD_train_efr.json")
#dataset_path = dataset_folder.joinpath('/MELD_train_efr.json')
#dataset_folder = "/kaggle/input/plaplapla/MELD_train_efr.json"
df = pd.read_json(dataset_folder)
#df['triggers'] = df['triggers'].fillna(value=0, inplace=False)#.replace('None', 0.0)

In [6]:
df

,episode,speakers,emotions,utterances,triggers
0,utterance_0,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 1.0, 0.0]"
1,utterance_1,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
2,utterance_2,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,utterance_3,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,utterance_4,"[Joey, Rachel, Joey, Rachel]","[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...
3995,utterance_3995,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3996,utterance_3996,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3997,utterance_3997,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3998,utterance_3998,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [7]:
triggers = df['triggers']
for row in range(triggers.shape[0]):
    for trigger in range(len(triggers[row])):
        if triggers[row][trigger] == None:
            triggers[row][trigger] = 0.0

df['triggers'] = triggers

In [8]:
emotions = df['emotions'].explode().unique()
emotions

array(['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust',
       'anger'], dtype=object)

In [9]:
triggers = df['triggers'].explode().unique()
triggers

array([0.0, 1.0], dtype=object)

In [10]:
dialogues = df['utterances']
#print(sentences)
max_len_dialogue = 0
index = 0
for idx, dialogue in enumerate(dialogues):
  if len(dialogue) > max_len_dialogue:
    max_len_dialogue = len(dialogue)
    index = idx
max_len_dialogue,index

(24, 219)

In [11]:
from sklearn.preprocessing import LabelBinarizer
sorted_emotions = sorted(emotions)  #sort the array because Binarizer will automatically do that for one hot encoding
label_binarizer = LabelBinarizer()
label_binarizer.fit(sorted_emotions)

dialogues = df['emotions']
one_hot_emotions = []
for dialogue_emotion in dialogues:
  dialogue_emotions_list = []
  for emotion in dialogue_emotion:
    encoded_emotion=label_binarizer.transform([emotion])
    dialogue_emotions_list.append(np.ravel(encoded_emotion).tolist())
  one_hot_emotions.append(dialogue_emotions_list)

In [12]:
df['emotions'] = one_hot_emotions

In [13]:
from sklearn.model_selection import train_test_split
train_data, temp_data = train_test_split(df, train_size=0.8, shuffle=False)
val_data, test_data = train_test_split(temp_data, test_size=0.5, shuffle=False)
val_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [14]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW, Trainer, TrainingArguments

class CustomDataset(Dataset):
    def __init__(self, dialogues, emotions, triggers, tokenizer, max_length=10):
        self.dialogues = dialogues
        self.emotions = emotions
        self.triggers = triggers
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
        dialogue = self.dialogues[idx]
        emotion = self.emotions[idx]
        trigger = self.triggers[idx]

        input_ids_list = []
        attention_mask_list = []

        for utterance in dialogue:
          tokenized_utterance = self.tokenizer(utterance, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
          # Extract relevant information
          #input_ids = torch.stack([inputs['input_ids'].squeeze() for inputs in tokenized_dialogue])
          input_ids_list.extend(tokenized_utterance['input_ids'])
          attention_mask_list.extend(tokenized_utterance['attention_mask'])

        emotion_labels = torch.tensor(emotion, dtype=torch.float32)
        trigger_label = torch.tensor(trigger, dtype=torch.long)
        #print('input',torch.stack(input_ids_list).shape)
        #print('attention',torch.stack(attention_mask_list).shape)
        #print('emotion',emotion_labels.shape)
        #print('trigeeer',trigger_label.shape)
        return {
            'input_ids': torch.stack(input_ids_list),
            'attention_mask': torch.stack(attention_mask_list),
            'emotion_labels': emotion_labels,
            'trigger_label': trigger_label
        }

In [61]:
# Baseline, no LSTM or Attention
import torch
from torch import nn
from transformers import XLNetModel

class CustomXLNetModel(nn.Module):
    def __init__(self, num_emotions, num_triggers):
        super(CustomXLNetModel, self).__init__()
        self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased')

        # Assuming we use the pooled output of XLNet for classification
        self.emotion_head = nn.Linear(self.xlnet.config.hidden_size, num_emotions)
        self.trigger_head = nn.Linear(self.xlnet.config.hidden_size, num_triggers)

    def forward(self, input_ids, attention_mask):
        xlnet_outputs = self.xlnet(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)

        # Get the last hidden state
        sequence_output = xlnet_outputs.last_hidden_state

        # Optionally, use the pooled output (representation of [CLS] token) as features for classification
        # However, note that XLNet does not provide a pooled output like BERT's `pooler_output`.
        # You might need to explicitly pool the last_hidden_state, for example, by taking the first token's representation or applying mean pooling.
        # Here, we take the representation of the first token as an example.
        pooled_output = sequence_output[:, 0]

        emotion_logits = self.emotion_head(pooled_output)
        trigger_logits = self.trigger_head(pooled_output)

        return emotion_logits, trigger_logits


In [54]:
# With Attention

import torch
from torch import nn
from transformers import XLNetModel

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attention_weights = nn.Parameter(torch.Tensor(hidden_size, 1))
        nn.init.xavier_uniform_(self.attention_weights.data)

    def forward(self, outputs):
        # Apply attention weights
        attention_scores = torch.matmul(outputs, self.attention_weights).squeeze(-1)
        attention_scores = torch.softmax(attention_scores, dim=1).unsqueeze(2)

        # Apply the attention scores to the outputs
        weighted_sequence = outputs * attention_scores
        attended_output = weighted_sequence.sum(dim=1)
        return attended_output

class CustomXLNetModel(nn.Module):
    def __init__(self, num_emotions, num_triggers):
        super(CustomXLNetModel, self).__init__()
        self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased')

        self.attention = Attention(self.xlnet.config.hidden_size)

        self.emotion_head = nn.Linear(self.xlnet.config.hidden_size, num_emotions)
        self.trigger_head = nn.Linear(self.xlnet.config.hidden_size, num_triggers)

    def forward(self, input_ids, attention_mask):
        xlnet_outputs = self.xlnet(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        sequence_output = xlnet_outputs.last_hidden_state

        attended_output = self.attention(sequence_output)

        emotion_logits = self.emotion_head(attended_output)
        trigger_logits = self.trigger_head(attended_output)

        return emotion_logits, trigger_logits


In [51]:
# With LSTM + Attention
import torch
from torch import nn
from transformers import XLNetModel

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attention_weights = nn.Parameter(torch.Tensor(hidden_size, 1))
        nn.init.xavier_uniform_(self.attention_weights.data)

    def forward(self, lstm_output):
        # Apply attention weights
        attention_scores = torch.matmul(lstm_output, self.attention_weights).squeeze(-1)
        attention_scores = torch.softmax(attention_scores, dim=1).unsqueeze(2)

        # Apply the attention scores to the lstm_output
        weighted_sequence = lstm_output * attention_scores
        attended_output = weighted_sequence.sum(dim=1)
        return attended_output

class CustomXLNetModel(nn.Module):
    def __init__(self, num_emotions, num_triggers):
        super(CustomXLNetModel, self).__init__()
        self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased')

        self.lstm_hidden_size = self.xlnet.config.hidden_size
        self.lstm = nn.LSTM(input_size=self.lstm_hidden_size,
                            hidden_size=self.lstm_hidden_size,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)

        self.attention = Attention(self.lstm_hidden_size * 2)

        self.emotion_head = nn.Linear(self.lstm_hidden_size * 2, num_emotions)
        self.trigger_head = nn.Linear(self.lstm_hidden_size * 2, num_triggers)

    def forward(self, input_ids, attention_mask):
        xlnet_outputs = self.xlnet(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        sequence_output = xlnet_outputs.last_hidden_state

        lstm_output, (h_n, c_n) = self.lstm(sequence_output)
        attended_output = self.attention(lstm_output)

        emotion_logits = self.emotion_head(attended_output)
        trigger_logits = self.trigger_head(attended_output)

        return emotion_logits, trigger_logits

In [72]:
from transformers import XLNetTokenizer

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

train_dataset = CustomDataset(train_data['utterances'], train_data['emotions'],
                              train_data['triggers'], tokenizer)
validation_dataset = CustomDataset(val_data['utterances'], val_data['emotions'],
                             val_data['triggers'], tokenizer)
test_dataset = CustomDataset(test_data['utterances'], test_data['emotions'],
                             test_data['triggers'], tokenizer)

# freezed_embeddings = False

custom_xlnet_model = CustomXLNetModel(num_emotions = 7, num_triggers = 2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_xlnet_model = custom_xlnet_model.to(device)
optimizer = AdamW(custom_xlnet_model.parameters(), lr=1e-5, weight_decay = 0)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [73]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    emotion_labels = [item['emotion_labels'] for item in batch]#torch.stack([item['emotion_labels'] for item in batch], dim=0)
    trigger_label = [item['trigger_label'] for item in batch]#torch.stack([item['trigger_label'] for item in batch], dim=0)

    #input_ids = pad_sequence([torch.stack(item['input_ids']) for item in batch], batch_first=True)
    #attention_mask = pad_sequence([torch.stack(item['attention_mask']) for item in batch], batch_first=True)
    return input_ids,attention_mask,emotion_labels,trigger_label
    #return {'input_ids': input_ids, 'attention_mask': attention_mask, 'emotion_labels': emotion_labels, 'trigger_label': trigger_label

In [67]:
def test_bert(mode,model_instance):

# Usage in the eval loop
  sequence_f1_scores_emotion = []
  sequence_f1_scores_trigger = []
  unrolled_predictions_emotion = []
  unrolled_predictions_trigger = []
  unrolled_labels_emotion = []
  unrolled_labels_trigger = []
  sequence_f1_scores = []

  batch_size = 1
  #test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
  if mode == 'validation':
    loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
  elif mode == 'test':
    loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
  # if model == 'baseline':
  #   BERT_baseline.eval()
  # elif model == 'custom':
  #   BERT_lstm.eval()
  model_instance.eval()
  with torch.no_grad():
      for batch in tqdm(loader, desc='Evaluation', leave=False):
          input_ids = batch['input_ids'].squeeze().to(device)
          attention_mask = batch['attention_mask'].squeeze().to(device)
          emotion_labels = batch['emotion_labels'].squeeze().to(device)
          trigger_label = batch['trigger_label'].squeeze().to(device)

          # if model == 'baseline':
          #   emotion_logits, trigger_logits = BERT_baseline(input_ids, attention_mask)
          # elif model == 'custom':
          #   emotion_logits, trigger_logits = BERT_lstm(input_ids, attention_mask)
          emotion_logits, trigger_logits = model_instance(input_ids, attention_mask)

          # Store predictions and labels for later unrolled F1 computation
          unrolled_predictions_emotion.append(emotion_logits)
          unrolled_labels_emotion.append(emotion_labels)
          unrolled_predictions_trigger.append(trigger_logits)
          unrolled_labels_trigger.append(trigger_label)

          # Convert logits to probabilities and then to class predictions
          #predicted_classes = torch.argmax(emotion_logits, dim=1)
          #true_classes = torch.argmax(emotion_labels, dim=1)

          # Compute F1 for the current sequence (dialogue)
          #sequence_f1 = f1_score(true_classes.cpu().numpy(), predicted_classes.cpu().numpy(), average='macro')
          #sequence_f1_scores.append(sequence_f1)

  # Compute the average Sequence F1 for emotions and triggers
  average_sequence_f1_emotion, average_sequence_f1_trigger = compute_sequence_f1(
      [unrolled_predictions_emotion, unrolled_predictions_trigger],
      [unrolled_labels_emotion, unrolled_labels_trigger]
  )

  # Compute the Unrolled Sequence F1 for emotions and triggers
  unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger = compute_unrolled_sequence_f1(
      [unrolled_predictions_emotion, unrolled_predictions_trigger],
      [unrolled_labels_emotion, unrolled_labels_trigger]
  )
  # if model == 'baseline':
  #   BERT_baseline.train()
  # elif model == 'custom':
  #   BERT_lstm.train()
  model_instance.train()
  # Print the F1 scores for emotions and triggers
  print(f"Average Sequence F1 (Emotion):  {average_sequence_f1_emotion:03f}")
  print(f"Average Sequence F1 (Trigger): {average_sequence_f1_trigger:03f}")
  print(f"Unrolled Sequence F1 (Emotion): {unrolled_sequence_f1_emotion.item():03f}")
  print(f"Unrolled Sequence F1 (Trigger): {unrolled_sequence_f1_trigger.item():03f}")
  return average_sequence_f1_emotion, average_sequence_f1_trigger, unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger

In [74]:
criterion_emotion = torch.nn.CrossEntropyLoss()
criterion_trigger = torch.nn.BCEWithLogitsLoss()
num_epochs = 5
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn = collate_fn)

In [75]:
for epoch in range(num_epochs):
    custom_xlnet_model.train()
    total_loss = 0.0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}', leave=False):
        input_ids,attention_mask,emotion_labels,trigger_label = batch

        # Zero the gradients on the optimizer
        optimizer.zero_grad()

        for el in range(batch_size):
          emotion_loss = 0.0
          trigger_loss = 0.0

          input_ids_el = input_ids[el].squeeze().to(device)
          attention_mask_el = attention_mask[el].squeeze().to(device)
          emotion_labels_el = emotion_labels[el].squeeze().to(device)
          trigger_label_el = trigger_label[el].squeeze().to(device)

          # Forward pass
          emotion_logits, trigger_logits = custom_xlnet_model(input_ids_el, attention_mask_el)
          # Compute the loss for both emotion and trigger

          emotion_loss += criterion_emotion(emotion_logits, torch.argmax(emotion_labels_el, dim=1))
          trigger_loss += criterion_trigger(torch.argmax(trigger_logits, dim=1).float(), trigger_label_el.float())

        # Combine losses for backpropagation
        loss = (emotion_loss + trigger_loss)/batch_size

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    # Compute the average loss
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {average_loss}")
    test_bert('validation', custom_xlnet_model)

Epoch 1, Average Loss: 0.08647044502198696


Average Sequence F1 (Emotion):  0.183634
Average Sequence F1 (Trigger): 0.330511
Unrolled Sequence F1 (Emotion): 0.085029
Unrolled Sequence F1 (Trigger): 0.334355


Epoch 2, Average Loss: 0.0829169850051403


Average Sequence F1 (Emotion):  0.183634
Average Sequence F1 (Trigger): 0.450336
Unrolled Sequence F1 (Emotion): 0.085029
Unrolled Sequence F1 (Trigger): 0.474217


Epoch 3, Average Loss: 0.08258714564144612


Average Sequence F1 (Emotion):  0.231562
Average Sequence F1 (Trigger): 0.462759
Unrolled Sequence F1 (Emotion): 0.137631
Unrolled Sequence F1 (Trigger): 0.509116


Epoch 4, Average Loss: 0.08187616527080536


Average Sequence F1 (Emotion):  0.280429
Average Sequence F1 (Trigger): 0.285186
Unrolled Sequence F1 (Emotion): 0.196562
Unrolled Sequence F1 (Trigger): 0.276280


Epoch 5, Average Loss: 0.08054202452301978


Average Sequence F1 (Emotion):  0.265491
Average Sequence F1 (Trigger): 0.307654
Unrolled Sequence F1 (Emotion): 0.178324
Unrolled Sequence F1 (Trigger): 0.306485


In [76]:
test_bert('test', custom_xlnet_model) # Baseline

Average Sequence F1 (Emotion):  0.300681
Average Sequence F1 (Trigger): 0.263727
Unrolled Sequence F1 (Emotion): 0.188067
Unrolled Sequence F1 (Trigger): 0.262602


(tensor(0.3007), tensor(0.2637), tensor(0.1881), tensor(0.2626))

Baseline BERT below


In [17]:
class CustomBERTModel(torch.nn.Module):
    def __init__(self, freeze_embeddings=True):
        super(CustomBERTModel, self).__init__()
        # Replace this with your custom BERT model architecture for multihead classification
        self.bert = BertModel.from_pretrained('bert-base-uncased')#BertForSequenceClassification.from_pretrained
        #LSTM
        if freeze_embeddings:
            for name,param in self.bert.named_parameters():
                if 'embeddings' in name:
                    param.requires_grad = False

        self.emotion_head = torch.nn.Linear(self.bert.config.hidden_size, len(emotions))
        self.trigger_head = torch.nn.Linear(self.bert.config.hidden_size, len(triggers))

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        pooled_output = outputs['pooler_output']

        # Emotion head
        emotion_logits = self.emotion_head(pooled_output)

        # Trigger head
        trigger_logits = self.trigger_head(pooled_output)
        return emotion_logits, trigger_logits
        #return emotion_outputs, trigger_outputs
        #return torch.stack(emotion_outputs), torch.stack(trigger_outputs)

In [19]:
# BERT baseline

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


trairn_dataset = CustomDataset(train_data['utterances'], train_data['emotions'],
                              train_data['triggers'], tokenizer)
validation_dataset = CustomDataset(val_data['utterances'], val_data['emotions'],
                             val_data['triggers'], tokenizer)
test_dataset = CustomDataset(test_data['utterances'], test_data['emotions'],
                             test_data['triggers'], tokenizer)


freezed_embeddings = False
custom_Bert_Model = CustomBERTModel(freezed_embeddings)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_Bert_Model = custom_Bert_Model.to(device)
optimizer = AdamW(filter(lambda p: p.requires_grad, custom_Bert_Model.parameters()), lr=5e-5)

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
from sklearn.metrics import f1_score

def compute_sequence_f1(predictions, labels):
    # predictions and labels should be lists of tensors for each dialogue
    emotion_f1_scores = []
    trigger_f1_scores = []
    for emotion_pred, trigger_pred, emotion_lab, trigger_lab in zip(predictions[0], predictions[1], labels[0], labels[1]):
        emotion_predicted_classes = torch.argmax(emotion_pred, dim=1)
        trigger_predicted_classes = torch.argmax(trigger_pred, dim=1)
        emotion_true_classes = torch.argmax(emotion_lab, dim=1)
        trigger_true_classes = trigger_lab
        emotion_f1 = f1_score(emotion_true_classes.cpu().numpy(), emotion_predicted_classes.cpu().numpy(), average='macro')
        trigger_f1 = f1_score(trigger_true_classes.cpu().numpy(), trigger_predicted_classes.cpu().numpy(), average='macro')
        emotion_f1_scores.append(emotion_f1)
        trigger_f1_scores.append(trigger_f1)
    average_emotion_f1 = torch.tensor(emotion_f1_scores, dtype=torch.float32).mean()
    average_trigger_f1 = torch.tensor(trigger_f1_scores, dtype=torch.float32).mean()
    return average_emotion_f1, average_trigger_f1

def compute_unrolled_sequence_f1(predictions, labels):
    # Flatten all utterances and compute the F1 score
    all_emotion_predicted_classes = torch.argmax(torch.cat(predictions[0], dim=0), dim=1)
    all_trigger_predicted_classes = torch.argmax(torch.cat(predictions[1], dim=0), dim=1)
    all_emotion_true_classes = torch.argmax(torch.cat(labels[0], dim=0), dim=1)
    all_trigger_true_classes = torch.cat(labels[1], dim=0)
    unrolled_emotion_f1 = f1_score(all_emotion_true_classes.cpu().numpy(), all_emotion_predicted_classes.cpu().numpy(), average='macro')
    unrolled_trigger_f1 = f1_score(all_trigger_true_classes.cpu().numpy(), all_trigger_predicted_classes.cpu().numpy(), average='macro')
    unrolled_emotion_f1_tensor = torch.tensor(unrolled_emotion_f1, dtype=torch.float32)
    unrolled_trigger_f1_tensor = torch.tensor(unrolled_trigger_f1, dtype=torch.float32)
    return unrolled_emotion_f1_tensor, unrolled_trigger_f1_tensor

## Bert

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
num_epochs = 3
batch_size = 1

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for epoch in range(num_epochs):
    custom_Bert_Model.train()
    total_loss = 0.0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}', leave=False):
        input_ids = batch['input_ids'].squeeze().to(device)
        attention_mask = batch['attention_mask'].squeeze().to(device)
        emotion_labels = batch['emotion_labels'].squeeze().to(device)
        trigger_label = batch['trigger_label'].squeeze().to(device)

        # Zero the gradients on the optimizer
        optimizer.zero_grad()

        # Forward pass
        emotion_logits, trigger_logits = custom_Bert_Model(input_ids, attention_mask)

        # Compute the loss for both emotion and trigger
        emotion_loss = criterion(emotion_logits, torch.argmax(emotion_labels, dim=1))
        trigger_loss = criterion(trigger_logits, trigger_label)

        # Combine losses for backpropagation
        loss = emotion_loss + trigger_loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Emotion loss:", emotion_loss)
    print(f"Trigger loss:", trigger_loss)
     # Compute the average loss
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {average_loss}")


Emotion loss: tensor(0.3195, device='cuda:0', grad_fn=<NllLossBackward0>)
Trigger loss: tensor(0.3524, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch 1, Average Loss: 1.473908871631138


Emotion loss: tensor(1.6035, device='cuda:0', grad_fn=<NllLossBackward0>)
Trigger loss: tensor(0.5599, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch 2, Average Loss: 0.9099623631429858


Emotion loss: tensor(0.6053, device='cuda:0', grad_fn=<NllLossBackward0>)
Trigger loss: tensor(0.2888, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch 3, Average Loss: 0.748000873948913


In [ ]:
# Usage in the eval loop
sequence_f1_scores_emotion = []
sequence_f1_scores_trigger = []
unrolled_predictions_emotion = []
unrolled_predictions_trigger = []
unrolled_labels_emotion = []
unrolled_labels_trigger = []
sequence_f1_scores = []

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluation', leave=False):
        input_ids = batch['input_ids'].squeeze().to(device)
        attention_mask = batch['attention_mask'].squeeze().to(device)
        emotion_labels = batch['emotion_labels'].squeeze().to(device)
        trigger_label = batch['trigger_label'].squeeze().to(device)

        emotion_logits, trigger_logits = custom_Bert_Model(input_ids, attention_mask)

        # Store predictions and labels for later unrolled F1 computation
        unrolled_predictions_emotion.append(emotion_logits)
        unrolled_labels_emotion.append(emotion_labels)
        unrolled_predictions_trigger.append(trigger_logits)
        unrolled_labels_trigger.append(trigger_label)

        # Convert logits to probabilities and then to class predictions
        predicted_classes = torch.argmax(emotion_logits, dim=1)
        true_classes = torch.argmax(emotion_labels, dim=1)

        # Compute F1 for the current sequence (dialogue)
        sequence_f1 = f1_score(true_classes.cpu().numpy(), predicted_classes.cpu().numpy(), average='macro')
        sequence_f1_scores.append(sequence_f1)

# Compute the average Sequence F1 for emotions and triggers
average_sequence_f1_emotion, average_sequence_f1_trigger = compute_sequence_f1(
    [unrolled_predictions_emotion, unrolled_predictions_trigger],
    [unrolled_labels_emotion, unrolled_labels_trigger]
)

# Compute the Unrolled Sequence F1 for emotions and triggers
unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger = compute_unrolled_sequence_f1(
    [unrolled_predictions_emotion, unrolled_predictions_trigger],
    [unrolled_labels_emotion, unrolled_labels_trigger]
)

# Print the F1 scores for emotions and triggers
print(f"Average Sequence F1 (Emotion): {average_sequence_f1_emotion}")
print(f"Average Sequence F1 (Trigger): {average_sequence_f1_trigger}")
print(f"Unrolled Sequence F1 (Emotion): {unrolled_sequence_f1_emotion.item()}")
print(f"Unrolled Sequence F1 (Trigger): {unrolled_sequence_f1_trigger.item()}")

Average Sequence F1 (Emotion): 0.2413686364889145
Average Sequence F1 (Trigger): 0.45175039768218994
Unrolled Sequence F1 (Emotion): 0.1467110812664032
Unrolled Sequence F1 (Trigger): 0.4606205224990845


## Majority classifier

In [ ]:
import torch

def find_majority_class(train_loader):
    # Initialize counters
    emotion_counts = torch.zeros(7) # Assuming there are 7 unique emotions
    trigger_counts = torch.zeros(2) # There are 2 classes for triggers: present or not
    negative_trigger_counts = 0
    positive_trigger_counts = 0
    # Iterate over the training dataset to count the labels
    for batch in train_loader:
        emotion_labels = batch['emotion_labels'].squeeze()
        trigger_labels = batch['trigger_label'].squeeze()
        #print(trigger_labels,torch.sum(trigger_labels, dim=0),(trigger_labels == 0).sum())
        # Sum up the counts for each class
        positive_trigger_counts += torch.sum(trigger_labels, dim=0)
        # Count the zeros for the negative class (absence of a trigger)
        # Since one-hot encoding, the absence is just the inverse of the presence
        negative_trigger_counts += torch.sum(1 - trigger_labels, dim=0)
        emotion_counts += torch.sum(emotion_labels, dim=0)

    trigger_counts[0] = negative_trigger_counts
    trigger_counts[1] = positive_trigger_counts
    print(trigger_counts)
    print(emotion_counts)
    # Find the index with the maximum count for emotions and triggers
    majority_emotion = torch.zeros_like(emotion_counts)
    majority_emotion[torch.argmax(emotion_counts)] = 1
    majority_trigger = torch.zeros_like(trigger_counts)
    majority_trigger[torch.argmax(trigger_counts)] = 1

    return majority_emotion, majority_trigger

# Let's assume that 'train_loader' is a DataLoader for your training dataset
# You need to replace 'train_loader' with the actual DataLoader for your dataset
majority_emotion, majority_trigger = find_majority_class(train_loader)
majority_emotion, majority_trigger

tensor([23524.,  4289.])
tensor([ 3025.,   816.,   917.,  5123., 12228.,  1929.,  3775.])


(tensor([0., 0., 0., 0., 1., 0., 0.]), tensor([1., 0.]))

In [ ]:
from sklearn.metrics import f1_score
from tqdm.auto import tqdm

def majority_classifier(majority_emotion, majority_trigger, test_loader):
    all_emotion_predictions = []
    all_trigger_predictions = []
    all_emotion_labels = []
    all_trigger_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Evaluation', leave=False):
            emotion_labels = batch['emotion_labels']
            trigger_labels = batch['trigger_label']

            for emotion_lab, trigger_lab in zip(emotion_labels, trigger_labels):
                # Ensure we have at least 1 dimension
                if emotion_lab.ndim == 1 and emotion_lab.size(0) == 1:
                    emotion_lab = emotion_lab.unsqueeze(0)
                if trigger_lab.ndim == 1 and trigger_lab.size(0) == 1:
                    trigger_lab = trigger_lab.unsqueeze(0)

                # Store the labels
                all_emotion_labels.append(emotion_lab)
                all_trigger_labels.append(trigger_lab)

                # Repeat the majority class prediction to match the number of utterances
                emotion_predictions = majority_emotion.repeat(emotion_lab.size(0), 1)
                #print(emotion_predictions)

                trigger_predictions = majority_trigger.repeat(trigger_lab.size(0), 1)

                # Store the predictions
                all_emotion_predictions.append(emotion_predictions)
                all_trigger_predictions.append(trigger_predictions)

    # Use the stored predictions and labels to calculate sequence F1 and unrolled sequence F1
    average_sequence_f1_emotion, average_sequence_f1_trigger = compute_sequence_f1(
        [all_emotion_predictions, all_trigger_predictions],
        [all_emotion_labels, all_trigger_labels]
    )

    unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger = compute_unrolled_sequence_f1(
        [all_emotion_predictions, all_trigger_predictions],
        [all_emotion_labels, all_trigger_labels]
    )

    return average_sequence_f1_emotion, average_sequence_f1_trigger, unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger

# Assume majority_emotion and majority_trigger are tensors of the majority class (one-hot encoded)
# and test_loader is your DataLoader instance for the test dataset.
average_f1_emotion, average_f1_trigger, unrolled_f1_emotion, unrolled_f1_trigger = majority_classifier(majority_emotion, majority_trigger, test_loader)

print(f"Average Sequence F1 (Emotion): {average_f1_emotion}")
print(f"Average Sequence F1 (Trigger): {average_f1_trigger}")
print(f"Unrolled Sequence F1 (Emotion): {unrolled_f1_emotion}")
print(f"Unrolled Sequence F1 (Trigger): {unrolled_f1_trigger}")

Evaluation:   0%|          | 0/400 [00:00<?, ?it/s]

Average Sequence F1 (Emotion): 0.1836344599723816
Average Sequence F1 (Trigger): 0.0
Unrolled Sequence F1 (Emotion): 0.08502866327762604
Unrolled Sequence F1 (Trigger): 0.0


## Random

In [ ]:
import torch

def random_classifier(test_loader, emotion_distribution, trigger_distribution):
    all_emotion_predictions = []
    all_trigger_predictions = []
    all_emotion_labels = []
    all_trigger_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Evaluation', leave=False):
            emotion_labels = batch['emotion_labels']
            trigger_labels = batch['trigger_label']

            for emotion_lab, trigger_lab in zip(emotion_labels, trigger_labels):
                # Ensure we have at least 1 dimension
                if emotion_lab.ndim == 1 and emotion_lab.size(0) == 1:
                    emotion_lab = emotion_lab.unsqueeze(0)
                if trigger_lab.ndim == 1 and trigger_lab.size(0) == 1:
                    trigger_lab = trigger_lab.unsqueeze(0)

                # Store the labels
                all_emotion_labels.append(emotion_lab)
                all_trigger_labels.append(trigger_lab)

                # Generate random predictions for emotions
                random_emotion_predictions = torch.randint(0, 2, (emotion_lab.size(0), 7))  # Randomly 0 or 1 for each emotion
                all_emotion_predictions.append(random_emotion_predictions.float())

                # Generate random predictions for triggers based on the training distribution
                random_trigger_probs = torch.rand((trigger_lab.size(0), 1))
                random_trigger_predictions = (random_trigger_probs < trigger_distribution).long()  # Binary prediction based on distribution
                random_trigger_predictions = torch.cat((random_trigger_predictions, 1 - random_trigger_predictions), dim=1)  # Make it one-hot
                all_trigger_predictions.append(random_trigger_predictions.float())

    # Calculate the F1 scores using your metric functions
    average_sequence_f1_emotion, average_sequence_f1_trigger = compute_sequence_f1(
        [all_emotion_predictions, all_trigger_predictions],
        [all_emotion_labels, all_trigger_labels]
    )

    unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger = compute_unrolled_sequence_f1(
        [all_emotion_predictions, all_trigger_predictions],
        [all_emotion_labels, all_trigger_labels]
    )

    return average_sequence_f1_emotion, average_sequence_f1_trigger, unrolled_sequence_f1_emotion, unrolled_sequence_f1_trigger

# You need to provide the distribution for the trigger class from your training data
# For example, if 30% of your training samples have a trigger, trigger_distribution should be 0.3
trigger_distribution = 0.5  # Replace with your actual distribution

# Now call your random classifier function
average_f1_emotion, average_f1_trigger, unrolled_f1_emotion, unrolled_f1_trigger = random_classifier(
    test_loader,
    emotion_distribution=None,  # Not used currently as we're assuming a uniform distribution
    trigger_distribution=trigger_distribution
)

print(f"Random Classifier Average Sequence F1 (Emotion): {average_f1_emotion}")
print(f"Random Classifier Average Sequence F1 (Trigger): {average_f1_trigger}")
print(f"Random Classifier Unrolled Sequence F1 (Emotion): {unrolled_f1_emotion}")
print(f"Random Classifier Unrolled Sequence F1 (Trigger): {unrolled_f1_trigger}")

Evaluation:   0%|          | 0/400 [00:00<?, ?it/s]

Random Classifier Average Sequence F1 (Emotion): 0.06286460161209106
Random Classifier Average Sequence F1 (Trigger): 0.29223355650901794
Random Classifier Unrolled Sequence F1 (Emotion): 0.07199179381132126
Random Classifier Unrolled Sequence F1 (Trigger): 0.29306313395500183
